In [13]:
from flask import Flask, request, jsonify
import os
import serpapi
import pandas as pd
import tensorflow as tf
from keras.preprocessing import image
import numpy as np

import logging
from PIL import Image
import io

In [25]:
app = Flask(__name__)

logging.basicConfig(level=logging.INFO)
# Load dataset
train_path = "C:/Users/rashe/ImageRecognitionProject/Groceries-Images-Recognition/ML/dataset/train"
image_category = os.listdir(train_path)

# Apply the suitable Image Preprocessing by using ImageDataGenerator
train_generator = ImageDataGenerator(rescale=1.0/255.0)
train_image_generator = train_generator.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Load the trained model
model = tf.keras.models.load_model('C:/Users/rashe/ImageRecognitionProject/Groceries-Images-Recognition/ML/mymodel.h5')

# Define the class map for label mapping
class_map = dict([v, k] for k, v in train_image_generator.class_indices.items())

@app.route('/')
def index():
    return 'Image Recognition API'


@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the image from the POST request
        img_file = request.files['image']
        
        # Check if the file is an image
        if img_file.filename == '':
            return jsonify({"error": "No image provided"}), 400
        
        # Load and preprocess the image
        img = Image.open(io.BytesIO(img_file.read()))
        img = img.resize((224, 224))  # Resize to match the model's input shape
        img_array = img_to_array(img) / 255.0
        img_input = img_array.reshape((1, img_array.shape[0], img_array.shape[1], img_array.shape[2]))

        # Debug: Print image shape
        print("Image shape:", img_input.shape)

        # Make prediction
        predictions = model.predict(img_input)
        print("Raw predictions:", predictions)

        predicted_label = int(predictions.argmax(axis=-1))
        print("Predicted label index:", predicted_label)
        
        # Debug: Print class map
        print("Class map:", class_map)

        predicted_img = class_map[predicted_label]

        # Return the result
        return jsonify({"predicted_product": predicted_img})

    except Exception as e:
        logging.error(f"Error during prediction: {e}")
        return jsonify({"error": "Failed to predict"}), 500

if __name__ == '__main__':
    app.run(debug=False)

Found 3115 images belonging to 36 classes.


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 02:45:43] "GET / HTTP/1.1" 200 -


Image shape: (1, 224, 224, 3)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 686ms/step


...

Raw predictions: [[9.8492730e-01 1.8479063e-10 8.9700916e-06 4.7654879e-08 3.0239292e-07
  2.0127224e-08 3.1799292e-08 6.2774512e-12 1.1077172e-08 8.5408560e-09
  3.3701291e-08 7.9225765e-07 3.6099341e-09 3.8940557e-09 9.0002157e-09
  2.7638310e-07 8.6515655e-08 4.9440119e-09 1.3748647e-06 3.2955990e-07
  1.0851540e-04 3.1491797e-06 3.2693848e-07 9.6849377e-05 2.9650164e-11
  4.1913681e-09 1.4667867e-02 2.0947542e-07 7.0310336e-05 1.0321035e-09
  1.4287204e-09 4.3175432e-09 1.9400037e-08 2.5251877e-05 7.7740326e-05
  1.0044303e-05]]
Predicted label index: 0
Class map: {0: 'apple', 1: 'banana', 2: 'beetroot', 3: 'bell pepper', 4: 'cabbage', 5: 'capsicum', 6: 'carrot', 7: 'cauliflower', 8: 'chilli pepper', 9: 'corn', 10: 'cucumber', 11: 'eggplant', 12: 'garlic', 13: 'ginger', 14: 'grapes', 15: 'jalepeno', 16: 'kiwi', 17: 'lemon', 18: 'lettuce', 19: 'mango', 20: 'onion', 21: 'orange', 22: 'paprika', 23: 'pear', 24: 'peas', 25: 'pineapple', 26: 'pomegranate', 27: 'potato', 28: 'raddish', 2

INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 02:46:28] "POST /predict HTTP/1.1" 200 -


Raw predictions: [[9.95722055e-01 2.20227765e-08 2.25473627e-07 4.47834100e-06
  5.05728082e-08 1.67722610e-05 6.42066595e-07 1.17764007e-10
  1.17139400e-07 1.59036091e-08 3.71736633e-08 7.33637280e-05
  6.69572731e-10 1.39673356e-10 5.31057731e-09 4.60568044e-05
  2.12793012e-07 1.02425616e-07 1.21585663e-05 3.10372712e-07
  2.73124315e-05 4.04245185e-07 1.39529384e-05 1.13390030e-04
  1.02468015e-07 3.84902599e-09 8.68248389e-06 8.47421067e-09
  1.75747991e-05 4.70692356e-08 1.54227759e-08 7.52083906e-08
  6.11076967e-09 3.94077692e-03 3.79283790e-07 5.61549882e-07]]
Predicted label index: 0
Class map: {0: 'apple', 1: 'banana', 2: 'beetroot', 3: 'bell pepper', 4: 'cabbage', 5: 'capsicum', 6: 'carrot', 7: 'cauliflower', 8: 'chilli pepper', 9: 'corn', 10: 'cucumber', 11: 'eggplant', 12: 'garlic', 13: 'ginger', 14: 'grapes', 15: 'jalepeno', 16: 'kiwi', 17: 'lemon', 18: 'lettuce', 19: 'mango', 20: 'onion', 21: 'orange', 22: 'paprika', 23: 'pear', 24: 'peas', 25: 'pineapple', 26: 'pomegr

INFO:werkzeug:127.0.0.1 - - [15/Apr/2024 02:46:59] "POST /predict HTTP/1.1" 200 -


Raw predictions: [[3.1546722e-03 1.0087010e-06 8.7882954e-01 4.8693331e-05 2.4598910e-08
  1.2764890e-03 7.7664270e-05 1.8789196e-07 1.3276942e-04 6.3770568e-07
  5.6502876e-07 2.9245156e-03 2.5193478e-06 2.2593399e-07 2.5216339e-06
  2.7361777e-05 5.5833522e-07 4.9199422e-09 6.3438830e-07 9.2461261e-05
  2.1373476e-03 2.0205252e-05 2.1297471e-03 8.9483353e-04 1.8857303e-07
  3.1357854e-06 2.8095173e-04 9.3185548e-07 6.3070878e-02 2.5681090e-06
  1.9515930e-06 1.5260702e-07 1.4901351e-06 4.3864708e-02 1.0165193e-03
  1.2838543e-06]]
Predicted label index: 2
Class map: {0: 'apple', 1: 'banana', 2: 'beetroot', 3: 'bell pepper', 4: 'cabbage', 5: 'capsicum', 6: 'carrot', 7: 'cauliflower', 8: 'chilli pepper', 9: 'corn', 10: 'cucumber', 11: 'eggplant', 12: 'garlic', 13: 'ginger', 14: 'grapes', 15: 'jalepeno', 16: 'kiwi', 17: 'lemon', 18: 'lettuce', 19: 'mango', 20: 'onion', 21: 'orange', 22: 'paprika', 23: 'pear', 24: 'peas', 25: 'pineapple', 26: 'pomegranate', 27: 'potato', 28: 'raddish', 2